In [2]:
import pandas as pd
import re

# Lendo o arquivo
with open("svr_results.txt", "r") as f:
    linhas = f.readlines()

# Lista para armazenar os dados
dados = []

# # Expressão regular para extrair os valores
# padrao = r"C=(\S+), epsilon=(\S+), gamma=(\S+), kernel=(\S+);, score=(-?\d+\.\d+) total time=\s+(\d+\.\d+)s"

# for linha in linhas:
#     match = re.search(padrao, linha)
#     if match:
#         dados.append(match.groups())
#     else:
#         print(f"Erro: {linha}")

# Expressão regular corrigida para capturar corretamente o tempo, independentemente do formato
padrao = r"C=(\S+), epsilon=(\S+), gamma=(\S+), kernel=(\S+);, score=(-?\d+\.\d+) total time=\s*([\d.]+)(s|min)"

for linha in linhas:
    match = re.search(padrao, linha)
    if match:
        C, epsilon, gamma, kernel, score, tempo, unidade = match.groups()
        tempo = float(tempo) * 60 if unidade == "min" else float(tempo)  # Convertendo minutos para segundos
        dados.append((float(C), float(epsilon), gamma, kernel, float(score), tempo))
    else:
        print(f"Erro: {linha}")


# Criando DataFrame
df = pd.DataFrame(dados, columns=["C", "epsilon", "gamma", "kernel", "score", "time"])

# Convertendo os tipos numéricos
df[["C", "epsilon", "score", "time"]] = df[["C", "epsilon", "score", "time"]].astype(float)

print(df)

        C  epsilon  gamma kernel  score  time
0    10.0    0.100  scale    rbf -0.070   1.5
1    10.0    0.100  scale    rbf -0.058   4.0
2     0.1    0.500  scale   poly -0.358   0.0
3     0.1    0.500  scale   poly -0.355   0.0
4     0.1    0.500  scale   poly -0.366   0.0
..    ...      ...    ...    ...    ...   ...
186   0.1    1.000   auto   poly -0.363   0.0
187   0.1    0.001   auto    rbf -0.035  13.2
188   0.1    0.001   auto    rbf -0.037  30.4
189   0.1    0.001   auto    rbf -0.044  50.5
190   0.1    0.001   auto    rbf -0.033  78.0

[191 rows x 6 columns]


In [3]:
# groupby com mean para score e time, ordenando por maior score
df_grouped = (
    df.groupby(["C", "epsilon", "gamma", "kernel"])
    .mean()
    .sort_values(by="score", ascending=False)
)

df_grouped

score      time
C      epsilon gamma kernel                         
10.0   0.001   auto  rbf         -0.035000    74.600
0.1    0.001   auto  rbf         -0.037250    43.025
1.0    0.001   scale rbf         -0.037600   200.740
10.0   0.010   auto  poly        -0.041800    20.540
1.0    0.001   scale poly        -0.042600  2715.800
       0.010   scale poly        -0.044800   545.980
0.1    0.010   auto  sigmoid     -0.045000    35.920
10.0   0.100   auto  rbf         -0.052200     5.120
               scale rbf         -0.060000    12.980
0.1    0.100   scale poly        -0.060000     6.200
1.0    0.100   scale poly        -0.064200    21.260
1000.0 0.010   scale rbf         -0.072750  8370.000
0.1    0.010   auto  poly        -0.074600    24.860
1.0    0.100   auto  poly        -0.082600     6.340
1000.0 0.100   scale poly        -0.101333  1256.000
0.1    0.100   auto  poly        -0.110600    11.200
1000.0 0.500   auto  sigmoid     -0.360600     0.000
       1.000   scale rbf         -0.360600     0.000
100.0  1.000   auto  rbf         -0.360600     0.000
       0.500   scale sigmoid     -0.360600     0.000
                     poly        -0.360600     0.000
10.0   0.500   scale poly        -0.360600     0.000
               auto  rbf         -0.360600     0.000
1000.0 0.500   auto  poly        -0.360600     0.000
       1.000   scale sigmoid     -0.360600     0.000
1.0    1.000   scale sigmoid     -0.360600     0.000
                     rbf         -0.360600     0.000
       0.500   auto  poly        -0.360600     0.000
0.1    1.000   scale sigmoid     -0.360600     0.000
               auto  sigmoid     -0.360600     0.000
                     poly        -0.360600     0.000
       0.500   scale rbf         -0.360600     0.000
                     poly        -0.360600     0.000
               auto  rbf         -0.360600     0.000
10.0   0.001   auto  sigmoid     -1.795400    53.800
1000.0 0.010   auto  sigmoid   -181.894200    45.700
10.0   0.010   scale sigmoid   -654.659200    46.200
100.0  0.100   scale sigmoid  -6546.196800    52.900
1000.0 0.010   scale sigmoid -65461.724400    51.160

In [5]:
df_grouped_2 = (
    df.groupby(["C", "epsilon", "gamma", "kernel"])
    .agg(score_mean=("score", "mean"), score_count=("score", "count"), time_mean=("time", "mean"))
    .sort_values(by="score_mean", ascending=False)
)

df_grouped_2

score_mean  score_count  time_mean
C      epsilon gamma kernel                                       
10.0   0.001   auto  rbf         -0.035000            5     74.600
0.1    0.001   auto  rbf         -0.037250            4     43.025
1.0    0.001   scale rbf         -0.037600            5    200.740
10.0   0.010   auto  poly        -0.041800            5     20.540
1.0    0.001   scale poly        -0.042600            5   2715.800
       0.010   scale poly        -0.044800            5    545.980
0.1    0.010   auto  sigmoid     -0.045000            5     35.920
10.0   0.100   auto  rbf         -0.052200            5      5.120
               scale rbf         -0.060000            5     12.980
0.1    0.100   scale poly        -0.060000            5      6.200
1.0    0.100   scale poly        -0.064200            5     21.260
1000.0 0.010   scale rbf         -0.072750            4   8370.000
0.1    0.010   auto  poly        -0.074600            5     24.860
1.0    0.100   auto  poly        -0.082600            5      6.340
1000.0 0.100   scale poly        -0.101333            3   1256.000
0.1    0.100   auto  poly        -0.110600            5     11.200
1000.0 0.500   auto  sigmoid     -0.360600            5      0.000
       1.000   scale rbf         -0.360600            5      0.000
100.0  1.000   auto  rbf         -0.360600            5      0.000
       0.500   scale sigmoid     -0.360600            5      0.000
                     poly        -0.360600            5      0.000
10.0   0.500   scale poly        -0.360600            5      0.000
               auto  rbf         -0.360600            5      0.000
1000.0 0.500   auto  poly        -0.360600            5      0.000
       1.000   scale sigmoid     -0.360600            5      0.000
1.0    1.000   scale sigmoid     -0.360600            5      0.000
                     rbf         -0.360600            5      0.000
       0.500   auto  poly        -0.360600            5      0.000
0.1    1.000   scale sigmoid     -0.360600            5      0.000
               auto  sigmoid     -0.360600            5      0.000
                     poly        -0.360600            5      0.000
       0.500   scale rbf         -0.360600            5      0.000
                     poly        -0.360600            5      0.000
               auto  rbf         -0.360600            5      0.000
10.0   0.001   auto  sigmoid     -1.795400            5     53.800
1000.0 0.010   auto  sigmoid   -181.894200            5     45.700
10.0   0.010   scale sigmoid   -654.659200            5     46.200
100.0  0.100   scale sigmoid  -6546.196800            5     52.900
1000.0 0.010   scale sigmoid -65461.724400            5     51.160